<a href="https://colab.research.google.com/github/ankesh86/PySparkNotebooks/blob/main/SupervisedLearningConcepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317122 sha256=617e00401c7cb406b24b8c1abfcec2dbac32aad8850d18468d54231ccd0475b1
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
#reading data
filename = "sample_data/bank-full.csv"

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read.csv(filename, header=True, inferSchema=True, sep=";")
data.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

# lendth of data

In [5]:
data.count()

45211

# Describe data

In [8]:
data.describe().toPandas()

,summary,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,count,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211
1,mean,40.93621021432837,None,None,None,None,1362.2720576850766,None,None,None,15.80641879188693,None,258.1630797814691,2.763840658246887,40.19782796222158,0.5803233726305546,None,None
2,stddev,10.618762040975408,None,None,None,None,3044.7658291685243,None,None,None,8.322476153044596,None,257.52781226517095,3.098020883279184,100.12874599059813,2.3034410449312204,None,None
3,min,18,admin.,divorced,primary,no,-8019,no,no,cellular,1,apr,0,1,-1,0,failure,no
4,max,95,unknown,single,unknown,yes,102127,yes,yes,unknown,31,sep,4918,63,871,275,unknown,yes


# Check Data types of each column

In [31]:
data.groupBy("marital").count().show()

+-------+-----+
|marital|count|
+-------+-----+
|    0.0|27214|
|    1.0|12790|
|    2.0| 5207|
+-------+-----+



In [10]:
data.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('y', 'string')]

In [11]:
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



# **Cardianility check**

In [13]:
from pyspark.sql.functions import approx_count_distinct, countDistinct

"""
Note: approxCountDistinct and countDistinct can be used interchangeably. Only difference is the computation time.

"approxCountDistinct" is useful for large datasets
"countDistinct" for small and medium datasets.

"""

def cardinality_calculation(df, cut_off=1):
    cardinality = df.select(*[approx_count_distinct(c).alias(c) for c in df.columns])

    ## convert to pandas for efficient calculations
    final_cardinality_df = cardinality.toPandas().transpose()
    final_cardinality_df.reset_index(inplace=True)
    final_cardinality_df.rename(columns={0:'Cardinality'}, inplace=True)

    #select variables with cardinality of 1
    vars_selected = final_cardinality_df['index'][final_cardinality_df['Cardinality'] <= cut_off]

    return final_cardinality_df, vars_selected

cardinality_df, cardinality_vars_selected = cardinality_calculation(data)

In [14]:
cardinality_df

,index,Cardinality
0,age,76
1,job,11
2,marital,3
3,education,4
4,default,2
5,balance,7375
6,housing,2
7,loan,2
8,contact,3
9,day,32


# **Missing value check**

In [15]:
#missing values check
from pyspark.sql.functions import count, when, isnan, col

# miss_percentage is set to 80% as discussed in the book
def missing_calculation(df, miss_percentage=0.80):

    #checks for both NaN and null values
    missing = df.select(*[count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
    length_df = df.count()
    ## convert to pandas for efficient calculations
    final_missing_df = missing.toPandas().transpose()
    final_missing_df.reset_index(inplace=True)
    final_missing_df.rename(columns={0:'missing_count'}, inplace=True)
    final_missing_df['missing_percentage'] = final_missing_df['missing_count']/length_df

    #select variables with cardinality of 1
    vars_selected = final_missing_df['index'][final_missing_df['missing_percentage'] >= miss_percentage]

    return final_missing_df, vars_selected

In [17]:
missing_calculation(data)

(        index  missing_count  missing_percentage
 0         age              0                 0.0
 1         job              0                 0.0
 2     marital              0                 0.0
 3   education              0                 0.0
 4     default              0                 0.0
 5     balance              0                 0.0
 6     housing              0                 0.0
 7        loan              0                 0.0
 8     contact              0                 0.0
 9         day              0                 0.0
 10      month              0                 0.0
 11   duration              0                 0.0
 12   campaign              0                 0.0
 13      pdays              0                 0.0
 14   previous              0                 0.0
 15   poutcome              0                 0.0
 16          y              0                 0.0,
 Series([], Name: index, dtype: object))

# **Identify variable types**

In [18]:
def variable_type(df):

    vars_list = df.dtypes
    char_vars = []
    num_vars = []
    for i in vars_list:
        if i[1] in ('string'):
            char_vars.append(i[0])
        else:
            num_vars.append(i[0])

    return char_vars, num_vars

In [19]:
char_vars, num_vars = variable_type(data)

In [20]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

def category_to_index(df, char_vars):

    char_df = df.select(char_vars)
    indexers = [StringIndexer(inputCol=c, outputCol=c+"_index", handleInvalid="keep") for c in char_df.columns]
    pipeline = Pipeline(stages=indexers)
    char_labels = pipeline.fit(char_df)
    df = char_labels.transform(df)
    return df, char_labels

data, char_labels = category_to_index(data, char_vars)

In [21]:
data.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('y', 'string'),
 ('job_index', 'double'),
 ('marital_index', 'double'),
 ('education_index', 'double'),
 ('default_index', 'double'),
 ('housing_index', 'double'),
 ('loan_index', 'double'),
 ('contact_index', 'double'),
 ('month_index', 'double'),
 ('poutcome_index', 'double'),
 ('y_index', 'double')]

In [22]:
data = data.select([c for c in data.columns if c not in char_vars])

In [23]:
def rename_columns(df, char_vars):
    mapping = dict(zip([i + '_index' for i in char_vars], char_vars))
    df = df.select([col(c).alias(mapping.get(c, c)) for c in df.columns])
    return df

In [24]:
data = rename_columns(data, char_vars)

In [25]:
data.dtypes

[('age', 'int'),
 ('balance', 'int'),
 ('day', 'int'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('job', 'double'),
 ('marital', 'double'),
 ('education', 'double'),
 ('default', 'double'),
 ('housing', 'double'),
 ('loan', 'double'),
 ('contact', 'double'),
 ('month', 'double'),
 ('poutcome', 'double'),
 ('y', 'double')]

In [26]:
data.groupBy('y').count().show()

+---+-----+
|  y|count|
+---+-----+
|0.0|39922|
|1.0| 5289|
+---+-----+



In [27]:
linear_df = data.select(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'])

In [29]:
linear_df

DataFrame[age: int, balance: int, day: int, duration: int, campaign: int, pdays: int, previous: int]

In [30]:
target_variable_name = 'balance'

# **Assemble input vectors**

In [32]:
#assemble feature vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

def assemble_vectors(df, features_list, target_variable_name):
  stages = []
  #assemble vectors
  assembler = VectorAssembler(inputCols=features_list, outputCol='features')
  stages = [assembler]

  #select all the columns + target + newly created "features" column
  selectedCols = [target_variable_name, 'features'] + features_list

  #using pipeline
  pipeline = Pipeline(stages=stages)

  #assembler model
  assembleModel = pipeline.fit(df)

  #assemble model on data
  df = assembleModel.transform(df).select(selectedCols)

  return df


In [36]:
#excluding target variable
features_list = linear_df.columns
#features_list = char_vars #this option is used only for ChiSqselector
features_list.remove(target_variable_name)

In [37]:
features_list

['age', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [38]:
# applying function on dataframe
df = assemble_vectors(linear_df, features_list, target_variable_name)


In [40]:
df.show()

+-------+--------------------+---+---+--------+--------+-----+--------+
|balance|            features|age|day|duration|campaign|pdays|previous|
+-------+--------------------+---+---+--------+--------+-----+--------+
|   2143|[58.0,5.0,261.0,1...| 58|  5|     261|       1|   -1|       0|
|     29|[44.0,5.0,151.0,1...| 44|  5|     151|       1|   -1|       0|
|      2|[33.0,5.0,76.0,1....| 33|  5|      76|       1|   -1|       0|
|   1506|[47.0,5.0,92.0,1....| 47|  5|      92|       1|   -1|       0|
|      1|[33.0,5.0,198.0,1...| 33|  5|     198|       1|   -1|       0|
|    231|[35.0,5.0,139.0,1...| 35|  5|     139|       1|   -1|       0|
|    447|[28.0,5.0,217.0,1...| 28|  5|     217|       1|   -1|       0|
|      2|[42.0,5.0,380.0,1...| 42|  5|     380|       1|   -1|       0|
|    121|[58.0,5.0,50.0,1....| 58|  5|      50|       1|   -1|       0|
|    593|[43.0,5.0,55.0,1....| 43|  5|      55|       1|   -1|       0|
|    270|[41.0,5.0,222.0,1...| 41|  5|     222|       1|   -1|  

# **Linear Regression**

In [41]:
# fit the regression model
from pyspark.ml.regression import LinearRegression
reg = LinearRegression(featuresCol='features', labelCol='balance')
reg_model = reg.fit(df) #fit model

In [43]:
# viewing coefficient and intercepts for each variable
import pandas as pd
for k,v in df.schema["features"].metadata["ml_attr"]["attrs"].items():
  features_df= pd.DataFrame(v)

#print coefficent and intercept
print(reg_model.coefficients, reg_model.intercept)
features_df['coefficients'] = reg_model.coefficients

[28.083972908930026,3.30554636194966,0.2488284197090184,-14.142676297161422,-0.08248810233031972,23.46299280076253] 124.9213009281818


In [44]:
features_df

,idx,name,coefficients
0,0,age,28.083973
1,1,day,3.305546
2,2,duration,0.248828
3,3,campaign,-14.142676
4,4,pdays,-0.082488
5,5,previous,23.462993


In [45]:
# prediction result
pred_result = reg_model.transform(df)

In [46]:
pred_result

DataFrame[balance: int, features: vector, age: int, day: int, duration: int, campaign: int, pdays: int, previous: int, prediction: double]

In [47]:
reg_model.summary.r2

0.010568116511551984

In [48]:
features_list

['age', 'day', 'duration', 'campaign', 'pdays', 'previous']

# **Variance Inflation factor**
To avoid multi-collinearity
Generally it should be less than 10 9acceptable), when near to 1, it not coorelated

In [50]:
def vif_calculator(df, features_list):
  vif_list = []
  for i in features_list:
    temp_features_list = features_list.copy()
    temp_features_list.remove(i)
    temp_target = i
    assembler = VectorAssembler(inputCols=temp_features_list, outputCol='features')
    temp_df = assembler.transform(df)

    reg = LinearRegression(featuresCol = 'features', labelCol = i)
    reg_model = reg.fit(temp_df) #fit model
    temp_vif = 1/(1-reg_model.summary.r2)
    vif_list.append(temp_vif)

  return vif_list

features_df['vif'] = vif_calculator(linear_df, features_list)
print(features_df)

   idx      name  coefficients       vif
0    0       age     28.083973  1.000917
1    1       day      3.305546  1.034350
2    2  duration      0.248828  1.007627
3    3  campaign    -14.142676  1.039907
4    4     pdays     -0.082488  1.276182
5    5  previous     23.462993  1.261321


# **Logistic Regression**

In [61]:
target_variable_name = "y"
logistic_df = data.select(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'y'])
#exclude target variable and select all other feature vectors
features_list = logistic_df.columns
#features_list = char_vars #this option is used only for ChiSqselector
features_list.remove(target_variable_name)
# apply the function on our dataframe
df = assemble_vectors(logistic_df, features_list, target_variable_name)

In [62]:
# Assuming df is your Spark DataFrame
unique_classes = df.select('y').distinct().collect()
print("Unique classes in 'y':", unique_classes)


Unique classes in 'y': [Row(y=0.0), Row(y=1.0)]


In [63]:
from pyspark.ml.classification import LogisticRegression

# Check the number of unique classes
num_classes = df.select('y').distinct().count()

# Initialize logistic regression according to the number of classes
if num_classes == 2:
    clf = LogisticRegression(featuresCol='features', labelCol='y', family='binomial')
elif num_classes > 2:
    clf = LogisticRegression(featuresCol='features', labelCol='y', family='multinomial')
else:
    raise ValueError("The target variable y must have at least two classes.")

# Fit the model
model = clf.fit(df)
print("Model fitted successfully with", ("binomial" if num_classes == 2 else "multinomial"), "family.")


IllegalArgumentException: requirement failed: Binomial family only supports 1 or 2 outcome classes but found 3.

In [57]:
from pyspark.ml.classification import LogisticRegression

# Initialize logistic regression
binary_clf = LogisticRegression(featuresCol='features', labelCol='y', family='binomial')
multinomial_clf = LogisticRegression(featuresCol='features', labelCol='y', family='multinomial')

# Fit the appropriate model based on the number of classes
if num_classes == 2:
    model = binary_clf.fit(df)
    print("Fitted binary logistic regression model")
elif num_classes > 2:
    model = multinomial_clf.fit(df)
    print("Fitted multinomial logistic regression model")
else:
    raise ValueError("The target variable y must have at least two classes for logistic regression.")


IllegalArgumentException: requirement failed: Binomial family only supports 1 or 2 outcome classes but found 3.

In [55]:
from pyspark.ml.classification import LogisticRegression
binary_clf = LogisticRegression(featuresCol='features', labelCol='y', family='binomial')
multinomial_clf = LogisticRegression(featuresCol='features', labelCol='y', family='multinomial')
binary_clf_model = binary_clf.fit(df) # fit binary model
multinomial_clf_model = multinomial_clf.fit(df) # fit multinomial model

IllegalArgumentException: requirement failed: Binomial family only supports 1 or 2 outcome classes but found 3.

In [56]:
# Count distinct values in the target variable
num_classes = logistic_df.select('y').distinct().count()
print(f"Number of classes: {num_classes}")


Number of classes: 2


In [59]:
df.show()

+---+--------------------+---+-------+---+--------+--------+-----+--------+
|  y|            features|age|balance|day|duration|campaign|pdays|previous|
+---+--------------------+---+-------+---+--------+--------+-----+--------+
|0.0|[58.0,2143.0,5.0,...| 58|   2143|  5|     261|       1|   -1|       0|
|0.0|[44.0,29.0,5.0,15...| 44|     29|  5|     151|       1|   -1|       0|
|0.0|[33.0,2.0,5.0,76....| 33|      2|  5|      76|       1|   -1|       0|
|0.0|[47.0,1506.0,5.0,...| 47|   1506|  5|      92|       1|   -1|       0|
|0.0|[33.0,1.0,5.0,198...| 33|      1|  5|     198|       1|   -1|       0|
|0.0|[35.0,231.0,5.0,1...| 35|    231|  5|     139|       1|   -1|       0|
|0.0|[28.0,447.0,5.0,2...| 28|    447|  5|     217|       1|   -1|       0|
|0.0|[42.0,2.0,5.0,380...| 42|      2|  5|     380|       1|   -1|       0|
|0.0|[58.0,121.0,5.0,5...| 58|    121|  5|      50|       1|   -1|       0|
|0.0|[43.0,593.0,5.0,5...| 43|    593|  5|      55|       1|   -1|       0|
|0.0|[41.0,2

IllegalArgumentException: Output column features already exists.